In [ ]:
from rdflib import Dataset, URIRef, Literal
from rdflib import Namespace
from rdflib.namespace import RDF, RDFS

In [ ]:
CRM = Namespace('http://cidoc-crm.org/cidoc-crm/')

In [ ]:
# the URI for our named graph that will contain our data
# CHANGE URI please to make it unique
#anno_named_graph_uri = 'http://example.org/ns/users/robcast/genesis-example'
anno_named_graph_uriref = URIRef(anno_named_graph_uri)

In [ ]:
# open a connection to the Blazegraph triple store
blazegraph_uri = 'http://localhost:8080/blazegraph/namespace/kb/sparql'
store = Dataset('SPARQLUpdateStore')
store.open((blazegraph_uri, blazegraph_uri))

In [ ]:
# find our named graph if it exists or create a new one
anno_named_graph = None
for g in store.graphs():
    if g.identifier == anno_named_graph_uriref:
        anno_named_graph = g

if anno_named_graph is None:
    anno_named_graph = store.graph(anno_named_graph_uriref)

In [ ]:
# find all annotations of type PERSON and extract the annotated name
persons = {}

res = anno_named_graph.query("""
prefix oa: <http://www.w3.org/ns/oa#>

SELECT ?anno ?name
WHERE {
  GRAPH <%s> {
    ?anno a oa:Annotation ;
      oa:hasBody / rdf:value "PERSON" ;
      oa:hasTarget / oa:hasSelector / oa:exact ?name .
  }
}
"""%(anno_named_graph_uri))

for r in res:
    name = str(r.name).replace(' ', '_')
    if name in persons:
        persons[name].append(r.anno)
    else:
        persons[name] = [r.anno]

In [ ]:
len(persons)

In [ ]:
# find all annotations of type River and extract the annotated name
rivers = {}

res = anno_named_graph.query("""
prefix oa: <http://www.w3.org/ns/oa#>

SELECT ?anno ?name
WHERE {
  GRAPH <%s> {
    ?anno a oa:Annotation ;
      oa:hasBody / rdf:value "River" ;
      oa:hasTarget / oa:hasSelector / oa:exact ?name .
  }
}
"""%(anno_named_graph_uri))

for r in res:
    name = str(r.name).replace(' ', '_')
    if name in rivers:
        rivers[name].append(r.anno)
    else:
        rivers[name] = [r.anno]

In [ ]:
len(rivers)

In [ ]:
# find all annotations of type Feature and extract the annotated name
features = {}

res = anno_named_graph.query("""
prefix oa: <http://www.w3.org/ns/oa#>

SELECT ?anno ?name
WHERE {
  GRAPH <%s> {
    ?anno a oa:Annotation ;
      oa:hasBody / rdf:value "PLACE" ;
      oa:hasTarget / oa:hasSelector / oa:exact ?name .
  }
}
"""%(anno_named_graph_uri))

for r in res:
    name = str(r.name).replace(' ', '_')
    if name in features:
        features[name].append(r.anno)
    else:
        features[name] = [r.anno]

In [ ]:
len(features)

In [ ]:
# create crm:E21_Person entries for each person
for name, annos in persons.items():
    person_uri = URIRef(f"{anno_named_graph_uri}/persons/{name}")
    # person_uri is a crm:E21_Person
    anno_named_graph.add((person_uri, RDF.type, CRM.E21_Person))
    # person has label name
    anno_named_graph.add((person_uri, RDFS.label, Literal(name)))
    
    for anno_uri in annos:
        # annotation_uri documents person_uri 
        anno_named_graph.add((anno_uri, CRM.E70_documents, person_uri))


In [ ]:
# create crm:E53_Place entries for each river
for name, annos in rivers.items():
    river_uri = URIRef(f"{anno_named_graph_uri}/rivers/{name}")
    # person_uri is a crm:E21_Person
    anno_named_graph.add((river_uri, RDF.type, CRM.E53_Place))
    # person has label name
    anno_named_graph.add((river_uri, RDFS.label, Literal(name)))
    
    for anno_uri in annos:
        # annotation_uri documents person_uri 
        anno_named_graph.add((anno_uri, CRM.E70_documents, river_uri))


In [ ]:
# create crm:E53_Place entries for each river
for name, annos in features.items():
    river_uri = URIRef(f"{anno_named_graph_uri}/places/{name}")
    # person_uri is a crm:E21_Person
    try:
        anno_named_graph.add((river_uri, RDF.type, CRM.E53_Place))
    except Exception as e:
        print(river_uri, e)
        pass
    # person has label name
    try:
        anno_named_graph.add((river_uri, RDFS.label, Literal(name)))
    except Exception as e: 
        print(river_uri, e)

    for anno_uri in annos:
        # annotation_uri documents person_uri 
        try:
            anno_named_graph.add((anno_uri, CRM.E70_documents, river_uri))
        except Exception as e:
            print(anno_uri, e)
            pass


In [ ]:
store.close()